In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import cftime

In [3]:
ds = xr.open_dataset('/glade/scratch/jinmuluo/MESOCOM/run/MESOCOM.clm2.h0.2010-03.nc')
ds

<xarray.Dataset>
Dimensions:                        (levgrnd: 25, levsoi: 20, levlak: 10, levdcmp: 25, time: 1, hist_interval: 2, lndgrid: 1, cft: 64, glc_nec: 10, ltype: 9, natpft: 15, nvegwcs: 4)
Coordinates:
  * levgrnd                        (levgrnd) float32 0.01 0.04 ... 28.87 42.0
  * levsoi                         (levsoi) float32 0.01 0.04 0.09 ... 6.94 8.03
  * levlak                         (levlak) float32 0.05 0.6 2.1 ... 34.33 44.78
  * levdcmp                        (levdcmp) float32 0.01 0.04 ... 28.87 42.0
  * time                           (time) object 2010-04-01 00:00:00
Dimensions without coordinates: hist_interval, lndgrid, cft, glc_nec, ltype, natpft, nvegwcs
Data variables: (12/561)
    mcdate                         (time) int32 ...
    mcsec                          (time) int32 ...
    mdcur                          (time) int32 ...
    mscur                          (time) int32 ...
    nstep                          (time) int32 ...
    time_bounds                    (time, hist_interval) object ...
    ...                             ...
    XSMRPOOL                       (time, lndgrid) float32 ...
    XSMRPOOL_RECOVER               (time, lndgrid) float32 ...
    ZBOT                           (time, lndgrid) float32 ...
    ZWT                            (time, lndgrid) float32 ...
    ZWT_CH4_UNSAT                  (time, lndgrid) float32 ...
    ZWT_PERCH                      (time, lndgrid) float32 ...
Attributes: (12/101)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              created on 08/25/23 09:09:08
    source:                               Community Terrestrial Systems Model
    hostname:                             cheyenne
    ...                                   ...
    cft_irrigated_tropical_corn:          62
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     month_1
    Time_constant_3Dvars_filename:        ./MESOCOM.clm2.h1.2010-01-01-00000.nc
    Time_constant_3Dvars:                 ZSOI:DZSOI:WATSAT:SUCSAT:BSW:HKSAT:...